In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import re #buscar, extraer o reemplazar patrones en cadenas de texto
import nltk #tokenización, stemming, análisis gramatical y clasificación de texto.
import matplotlib.pyplot as plt
from nltk.corpus import stopwords # libreria para el procesado de texto
from nltk.stem.porter import PorterStemmer #reducir palabras a su raíz (stemming); tareas como búsqueda o clasificación textual
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\david\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# mostramos que es Stopwords
print(stopwords.words('english')) # palabras más usadas del idioma

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [8]:
# Pre-procesado de datos

In [10]:
df=pd.read_csv(r'C:\Users\david\Desktop\Curso ML\Pandas\train.csv')

In [12]:
df.head() # label--> 1: fake new ; label-->0: real new

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
# comprobamos número de missing values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [16]:
# en este caso podemos sustituir los missing values por null string ('')
df=df.fillna('')

In [18]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [26]:
# creamos una nueva columna que incluya autor + title
df['content']=df['author']+' '+df['title']

In [30]:
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [32]:
# nos quedamos solo con content (feature) y label (target) 

In [36]:
X=df['content']
y=df['label'] # target

In [40]:
print(X)

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [42]:
# procesado de X - stemming - reducir la palabra a su key word (root) - funcion: PorterStemmer
port_stem=PorterStemmer()

In [44]:
# ejemplo sencillo:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
print(stemmer.stem("running")) # Devuelve "run" --> root word (raiz)

run


In [46]:
# stemming
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content) # Eliminar todos los caracteres que no sean letras minúscula o mayuscula 
    # El patrón [ ^a-zA-Z ] indica que se deben buscar todos los caracteres que NO sean letras del alfabeto
        # ^: Dentro de un conjunto ([]), significa "no incluir". Por lo tanto no incluir caracteres de a-z y de A-Z
    # Reemplazo con ' ' --> Todas las coincidencias encontradas por el patrón serán reemplazadas por un espacio (' ').
    # Esto asegura que todo carácter no alfabético sea eliminado y sustituido por un espacio
    stemmed_content=stemmed_content.lower() # conversion a minusculas
    stemmed_content=stemmed_content.split() # Convertir texto en lista de palabras
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not in stopwords.words('english')]
    #Elimina palabras comunes sin significado (the, and, or). Stemming reduce palabras a su raíz
    stemmed_content= ' '.join(stemmed_content) # une elemntos de la lista en un sola cadena:
    # uso--> elemento_separador.join(datos); Une todos los elementos de la lista stemmed_content en una sola cadena de texto.
    #Cada palabra está separada por un espacio (' '), que es el delimitador especificado.
    return stemmed_content

In [48]:
# aplicamos la funcion
df['content']=df['content'].apply(stemming)

In [50]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [52]:
X=df['content'].values # pasar de df o serie como un array numpy
y=df['label'].values

In [54]:
print(y)

[1 0 1 ... 0 1 1]


In [56]:
y.shape

(20800,)

In [58]:
# convertimos text data en vectores - Funcion Tf-idf
vectorizer=TfidfVectorizer()
X_vect=vectorizer.fit_transform(X)

In [62]:
print(X_vect) # con estos datos entrenamos nuestro modelo

  (0, 3600)	0.3598939188262559
  (0, 8909)	0.3635963806326075
  (0, 7005)	0.21874169089359144
  (0, 3792)	0.2705332480845492
  (0, 267)	0.27010124977708766
  (0, 4973)	0.233316966909351
  (0, 13473)	0.2565896679337957
  (0, 2959)	0.2468450128533713
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 2483)	0.3676519686797209
  (0, 15686)	0.28485063562728646
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 2813)	0.19094574062359204
  (1, 1497)	0.2939891562094648
  (1, 16799)	0.30071745655510157
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (2, 3103)	0.46097489583229645
  (2, 2943)	0.3179886800654691
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  :	:
  (20797, 9588)	0.17455348025522197
  (20797, 7042)	0.21799048897828685
  (20797, 3643)	0.2115550061362374
  (20797, 8364)	0.22322585870464115
  (20797, 9518)	0.29542040034203126
  (20797, 12138)	0.2477825

In [64]:
X_train,X_test,y_train,y_test=train_test_split(X_vect,y,test_size=0.2,random_state=2,stratify=y)

In [66]:
print(X_vect.shape,X_train.shape,X_test.shape)

(20800, 17128) (16640, 17128) (4160, 17128)


In [75]:
# Entrenamos el modelo
model=LogisticRegression()

In [77]:
model.fit(X_train,y_train)

LogisticRegression()

In [79]:
# Evaluamos el modelo

In [86]:
# Training data
X_train_pred=model.predict(X_train)
train_accuracy=accuracy_score(X_train_pred,y_train)
print(train_accuracy)

0.9863581730769231


In [88]:
# Testing data
X_test_pred=model.predict(X_test)
test_accuracy=accuracy_score(X_test_pred,y_test)
print(test_accuracy)

0.9790865384615385


In [90]:
# Predicting System

In [106]:
X_new=X_test[1] # tomamos el primer valor de los datos test para aplicar el modelo

prediction=model.predict(X_new)
print(prediction)

if prediction[0] ==0:
    print('Real new')
else:
    print('Fake new')

[0]
Real new


In [108]:
# vemos si es cierto
print(y_test[1]) # label asociado a X_test[0]

0
